# Lets create a Recommendation system for the Henry coefficients of porous materials

In [102]:
import json
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import pprint
import os
import pygaps
import scipy
from scipy.sparse.linalg import svds

## We will use the Henry Coefficient matrix, created in `HenryMatrix.ipynb`

In [103]:
henry_df = pd.read_csv('data/henry_matrix_median.csv')
henry_df.set_index(list(henry_df)[0], inplace=True)
henry_df.head(10)

,CURLTUGMZLYLDI-UHFFFAOYSA-N_298K,IJGRMHOSHXDMSA-UHFFFAOYSA-N_298K,VNWKTOKETHGBQD-UHFFFAOYSA-N_298K,MYMOFIZGZYHOMD-UHFFFAOYSA-N_298K,IJGRMHOSHXDMSA-UHFFFAOYSA-N_77K,CURLTUGMZLYLDI-UHFFFAOYSA-N_195K,UFHFLCQGNIYNRP-UHFFFAOYSA-N_77K,QQONPFPTGQHPMA-UHFFFAOYSA-N_298K,ATUOYWHBWRKTHZ-UHFFFAOYSA-N_298K,HSFWRNGVRCDJHI-UHFFFAOYSA-N_298K,...,WEVYAHXRMPXWCK-UHFFFAOYSA-N_298K,YXFVVABEGXRONW-UHFFFAOYSA-N_298K,RWSOTUBLDIXVET-UHFFFAOYSA-N_298K,OFBQJSOFQDEBGM-UHFFFAOYSA-N_298K,LRHPLDYGYMQRHN-UHFFFAOYSA-N_298K,URLKBWYHVLBVBO-UHFFFAOYSA-N_298K,RAHZWNYVWXNFOC-UHFFFAOYSA-N_298K,SWQJXJOGLNCZEY-UHFFFAOYSA-N_298K,VXNZUUAINFGPBY-UHFFFAOYSA-N_298K,QWTDNUCVQCZILF-UHFFFAOYSA-N_298K
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
NIST-MATDB-b2dc02871a66ac38e4b195796cf3e00b,0.643047,7.743846e-02,0.239119,NaN,1672.743597,NaN,160.335019,13.210848,18.987584,0.001506,...,NaN,NaN,NaN,NaN,NaN,184.515114,65.502827,NaN,NaN,NaN
NIST-MATDB-035dc75dddd00241bd76627f78cbef2d,73.595342,3.037941e-01,0.770303,9.101240e-02,3379.729460,NaN,NaN,52.095266,80.817935,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000432,96.786201,NaN
NIST-MATDB-453276ee992ed07dc207ed08b2b221ce,1.760340,2.719285e-01,0.326730,2.000000e-10,1227.679975,9.153744e+01,1208.198529,NaN,NaN,NaN,...,NaN,NaN,12.899584,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NIST-MATDB-1b85b1862d4e9cc2e0f46d0e2698ab79,2.172201,2.008086e-01,0.690172,1.826010e-01,1875.572443,NaN,112.247437,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2545.488306,NaN,NaN,NaN,NaN
NIST-MATDB-991daf7313251e7e607e2bab2da57e33,4.277288,2.162572e-01,0.545481,2.298642e-01,1857.126420,2.000000e-10,1000.396903,638.464909,48.655815,86.774658,...,NaN,NaN,NaN,5039.506517,NaN,NaN,94.944295,NaN,NaN,4128.514098
NIST-MATDB-9f02824618e94d86caedc9fb497366ba,0.226441,NaN,0.003670,NaN,1184.815284,NaN,850.010218,NaN,NaN,5.888601,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NIST-MATDB-acbbe796d49416e999dc819f2ae65fe9,3.040278,NaN,14.129264,NaN,2770.241788,NaN,1249.618229,NaN,NaN,51.060141,...,NaN,NaN,NaN,NaN,NaN,NaN,57.137198,NaN,NaN,NaN
NIST-MATDB-e23e0f3921a269533d74caae8c5a12bf,2.021117,2.000000e-10,33.817715,NaN,982.709651,3.430361e-01,1963.214804,NaN,NaN,NaN,...,NaN,NaN,NaN,1.402827,NaN,NaN,NaN,NaN,NaN,NaN
NIST-MATDB-9d023a670a2aedb5225d9cd365e24210,2.220104,NaN,0.551645,NaN,18491.161118,2.579633e+01,25.177850,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17.700403,NaN,NaN,NaN


## Lets also define a function that will randomly split our data into a testing and training set

In [98]:
# Will create a mask that will split the sparse matrix into two matrices. 
#  `train_frac` is the fraction of the data (excluding the NaN's) that will be used for training.
#  the fraction of the data used for testing will be `1 - train_frac`.
#
# train_matrix, test_matrix = split_matrix(henry_matrix, 0.8)
def split_matrix(henry_matrix, train_frac):
    matrix_size = np.size(henry_matrix)
    matrix_nan_size = np.sum(np.sum(np.isnan(henry_matrix)))
    henry_data_size = matrix_size - matrix_nan_size
    train_mask = np.zeros(henry_matrix.shape, dtype=bool)
    n_retained = np.floor(train_frac * henry_data_size)
    n_current = 0
    while True:
        i_material = random.choice(np.arange(len(henry_matrix.index)))
        j_gas = random.choice(np.arange(len(henry_matrix.columns)))
        if not np.isnan(henry_matrix.iat[i_material, j_gas]) and not train_mask[i_material, j_gas]:
            train_mask[i_material, j_gas] = True
            n_current += 1
        if n_current >= n_retained:
            break
    train_matrix = henry_matrix.mask(train_mask)
    test_matrix = henry_matrix.mask(~train_mask)
    #print(henry_data_size)
    #print(np.sum(np.sum(~np.isnan(train_matrix))) + np.sum(np.sum(~np.isnan(test_matrix))))
    assert henry_data_size == np.sum(np.sum(~np.isnan(train_matrix))) + np.sum(np.sum(~np.isnan(test_matrix)))
    return train_matrix, test_matrix
tr_m, te_m = split_matrix(henry_df, 0.8)

In [99]:
np.sum(np.sum(np.isnan(tr_m)))

11144

In [104]:
henry_df.head(10)

,CURLTUGMZLYLDI-UHFFFAOYSA-N_298K,IJGRMHOSHXDMSA-UHFFFAOYSA-N_298K,VNWKTOKETHGBQD-UHFFFAOYSA-N_298K,MYMOFIZGZYHOMD-UHFFFAOYSA-N_298K,IJGRMHOSHXDMSA-UHFFFAOYSA-N_77K,CURLTUGMZLYLDI-UHFFFAOYSA-N_195K,UFHFLCQGNIYNRP-UHFFFAOYSA-N_77K,QQONPFPTGQHPMA-UHFFFAOYSA-N_298K,ATUOYWHBWRKTHZ-UHFFFAOYSA-N_298K,HSFWRNGVRCDJHI-UHFFFAOYSA-N_298K,...,WEVYAHXRMPXWCK-UHFFFAOYSA-N_298K,YXFVVABEGXRONW-UHFFFAOYSA-N_298K,RWSOTUBLDIXVET-UHFFFAOYSA-N_298K,OFBQJSOFQDEBGM-UHFFFAOYSA-N_298K,LRHPLDYGYMQRHN-UHFFFAOYSA-N_298K,URLKBWYHVLBVBO-UHFFFAOYSA-N_298K,RAHZWNYVWXNFOC-UHFFFAOYSA-N_298K,SWQJXJOGLNCZEY-UHFFFAOYSA-N_298K,VXNZUUAINFGPBY-UHFFFAOYSA-N_298K,QWTDNUCVQCZILF-UHFFFAOYSA-N_298K
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
NIST-MATDB-b2dc02871a66ac38e4b195796cf3e00b,0.643047,7.743846e-02,0.239119,NaN,1672.743597,NaN,160.335019,13.210848,18.987584,0.001506,...,NaN,NaN,NaN,NaN,NaN,184.515114,65.502827,NaN,NaN,NaN
NIST-MATDB-035dc75dddd00241bd76627f78cbef2d,73.595342,3.037941e-01,0.770303,9.101240e-02,3379.729460,NaN,NaN,52.095266,80.817935,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000432,96.786201,NaN
NIST-MATDB-453276ee992ed07dc207ed08b2b221ce,1.760340,2.719285e-01,0.326730,2.000000e-10,1227.679975,9.153744e+01,1208.198529,NaN,NaN,NaN,...,NaN,NaN,12.899584,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NIST-MATDB-1b85b1862d4e9cc2e0f46d0e2698ab79,2.172201,2.008086e-01,0.690172,1.826010e-01,1875.572443,NaN,112.247437,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2545.488306,NaN,NaN,NaN,NaN
NIST-MATDB-991daf7313251e7e607e2bab2da57e33,4.277288,2.162572e-01,0.545481,2.298642e-01,1857.126420,2.000000e-10,1000.396903,638.464909,48.655815,86.774658,...,NaN,NaN,NaN,5039.506517,NaN,NaN,94.944295,NaN,NaN,4128.514098
NIST-MATDB-9f02824618e94d86caedc9fb497366ba,0.226441,NaN,0.003670,NaN,1184.815284,NaN,850.010218,NaN,NaN,5.888601,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NIST-MATDB-acbbe796d49416e999dc819f2ae65fe9,3.040278,NaN,14.129264,NaN,2770.241788,NaN,1249.618229,NaN,NaN,51.060141,...,NaN,NaN,NaN,NaN,NaN,NaN,57.137198,NaN,NaN,NaN
NIST-MATDB-e23e0f3921a269533d74caae8c5a12bf,2.021117,2.000000e-10,33.817715,NaN,982.709651,3.430361e-01,1963.214804,NaN,NaN,NaN,...,NaN,NaN,NaN,1.402827,NaN,NaN,NaN,NaN,NaN,NaN
NIST-MATDB-9d023a670a2aedb5225d9cd365e24210,2.220104,NaN,0.551645,NaN,18491.161118,2.579633e+01,25.177850,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17.700403,NaN,NaN,NaN


In [105]:
tr_m.head(10)

,CURLTUGMZLYLDI-UHFFFAOYSA-N_298K,IJGRMHOSHXDMSA-UHFFFAOYSA-N_298K,VNWKTOKETHGBQD-UHFFFAOYSA-N_298K,MYMOFIZGZYHOMD-UHFFFAOYSA-N_298K,IJGRMHOSHXDMSA-UHFFFAOYSA-N_77K,CURLTUGMZLYLDI-UHFFFAOYSA-N_195K,UFHFLCQGNIYNRP-UHFFFAOYSA-N_77K,QQONPFPTGQHPMA-UHFFFAOYSA-N_298K,ATUOYWHBWRKTHZ-UHFFFAOYSA-N_298K,HSFWRNGVRCDJHI-UHFFFAOYSA-N_298K,...,WEVYAHXRMPXWCK-UHFFFAOYSA-N_298K,YXFVVABEGXRONW-UHFFFAOYSA-N_298K,RWSOTUBLDIXVET-UHFFFAOYSA-N_298K,OFBQJSOFQDEBGM-UHFFFAOYSA-N_298K,LRHPLDYGYMQRHN-UHFFFAOYSA-N_298K,URLKBWYHVLBVBO-UHFFFAOYSA-N_298K,RAHZWNYVWXNFOC-UHFFFAOYSA-N_298K,SWQJXJOGLNCZEY-UHFFFAOYSA-N_298K,VXNZUUAINFGPBY-UHFFFAOYSA-N_298K,QWTDNUCVQCZILF-UHFFFAOYSA-N_298K
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
NIST-MATDB-b2dc02871a66ac38e4b195796cf3e00b,NaN,NaN,NaN,NaN,1672.743597,NaN,NaN,NaN,18.987584,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,65.502827,NaN,NaN,NaN
NIST-MATDB-035dc75dddd00241bd76627f78cbef2d,73.595342,0.303794,NaN,NaN,NaN,NaN,NaN,52.095266,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000432,96.786201,NaN
NIST-MATDB-453276ee992ed07dc207ed08b2b221ce,NaN,0.271928,NaN,NaN,NaN,NaN,1208.198529,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NIST-MATDB-1b85b1862d4e9cc2e0f46d0e2698ab79,2.172201,0.200809,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NIST-MATDB-991daf7313251e7e607e2bab2da57e33,NaN,NaN,NaN,NaN,NaN,NaN,1000.396903,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NIST-MATDB-9f02824618e94d86caedc9fb497366ba,NaN,NaN,NaN,NaN,1184.815284,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NIST-MATDB-acbbe796d49416e999dc819f2ae65fe9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NIST-MATDB-e23e0f3921a269533d74caae8c5a12bf,2.021117,NaN,33.817715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NIST-MATDB-9d023a670a2aedb5225d9cd365e24210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
